# 3. Machine Learning for Classification

In [451]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [452]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [453]:
#the ! indicate that we use a shell command
!wget $data
#to rename file
#!wget $data -O data-week-3.csv

--2023-10-01 10:51:47--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘WA_Fn-UseC_-Telco-Customer-Churn.csv.4’

WA_Fn-UseC_-Telco-C 100%[===================>] 954,59K  2,27MB/s    in 0,4s    

2023-10-01 10:51:48 (2,27 MB/s) - ‘WA_Fn-UseC_-Telco-Customer-Churn.csv.4’ saved [977501/977501]



In [454]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [455]:
# to see all column we can transpose the dataframe so that index becames columns and coolumns becom index
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [456]:
#make everything lowercase and replace  space with underscore
df.columns = df.columns.str.lower().str.replace(' ','_')

In [457]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

In [458]:
for col in categorical_columns:
    df[col] = df[col].str.lower().str.replace(' ','_')

In [459]:
df.dtypes
#totalcharges is object but should be integer
#seniorcitizen is number (can keep for mpdelling?)

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [460]:
# errors{‘ignore’, ‘raise’, ‘coerce’}, default ‘raise’
# If ‘raise’, then invalid parsing will raise an exception.

# If ‘coerce’, then invalid parsing will be set as NaN.

# If ‘ignore’, then invalid parsing will return the input.

pd.to_numeric(df.totalcharges, errors = 'coerce')

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: totalcharges, Length: 7043, dtype: float64

In [461]:
#make totalcharges numeric, put na when there is an error, fill na with 0
#NEW METHOD
df.totalcharges = pd.to_numeric(df.totalcharges, errors = 'coerce')

df.totalcharges = df.totalcharges.fillna(0)


In [462]:
#churn has yes and no values, replace with 1 and 0
#AMAZING way!
df.churn = (df.churn == 'yes').astype(int)

## 3.3 Setting up the validation framework (train, validation, test)

In [463]:
from sklearn.model_selection import train_test_split

In [464]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [465]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [466]:
len(df_train), len(df_test), len(df_val)

(4225, 1409, 1409)

In [467]:
df_train = df_train.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)

In [468]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [469]:
#delete from the dataframes so you do not accidentally keep them in the dataframe
del df_train['churn']
del df_val['churn']
del df_test['churn']

## 3.4 Exploratory Data Analysis

In [470]:
df_full_train = df_full_train.reset_index(drop = True)
#Checking for missing values
df_full_train.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [471]:
#look at distribution of churn variable

(df_full_train.churn == 1).sum(), (df_full_train.churn == 0).sum()


#Same AS

df_full_train.churn.value_counts()
df_full_train.churn.value_counts(normalize = True)

#26% churn rate!

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [472]:
#Here the mean also gives the churn rate, because  sum of all churn rate values == number of churns == number of ones
global_churn_rate = df_full_train.churn.mean()
round(global_churn_rate,2)

0.27

In [473]:
df_full_train.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges        float64
churn                 int64
dtype: object

In [474]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [475]:
df_full_train.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [476]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [477]:
df_full_train[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

## 3.5 Feature importance: Churn rate and risk ratio

tell us how important those categorical variables are

In [478]:
#Look at churn rate within different groups
df_full_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,5442-pptjy,male,0,yes,yes,12,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.70,258.35,0
1,6261-rcvns,female,0,no,no,42,yes,no,dsl,yes,...,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.90,3160.55,1
2,2176-osjuv,male,0,yes,no,71,yes,yes,dsl,yes,...,no,yes,no,no,two_year,no,bank_transfer_(automatic),65.15,4681.75,0
3,6161-erdgd,male,0,yes,yes,71,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,no,electronic_check,85.45,6300.85,0
4,2364-ufrom,male,0,no,no,30,yes,no,dsl,yes,...,no,yes,yes,no,one_year,no,electronic_check,70.40,2044.75,0


In [479]:
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_female

0.27682403433476394

In [480]:
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_male

0.2632135306553911

In [481]:
df_full_train.partner.value_counts()

partner
no     2932
yes    2702
Name: count, dtype: int64

In [482]:
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()
churn_no_partner

0.3298090040927694

In [483]:
churn_partner = df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_partner



0.20503330866025166

We see that for some catogircal variables the difference to the mean is bigger or smaller showing us what variables are important

### Feature Importance

1. Difference (Absolute Terms)

    Global Churn Rate - Group Churn Rate ( If negative --> more likely to churn, more interesting for us)


2. Risk Ratio (Relative Terms)

    Risk = Group/Global (If nigger than 1, more likely to churn --> more interesting for us)


In [484]:
from IPython.display import display

In [485]:
#NEW METHOD
for c  in categorical:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - df_full_train.churn.mean()
    df_group['risk'] = df_group['mean']/df_full_train.churn.mean()
    display(df_group)
    print()
    print()




gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006856,1.025396
male,0.263214,2838,-0.006755,0.974980




seniorcitizen


,mean,count,diff,risk
seniorcitizen,,,,
0,0.242270,4722,-0.027698,0.897403
1,0.413377,912,0.143409,1.531208




partner


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161733,1.599082
one_year,0.120573,1186,-0.149395,0.446621
two_year,0.028274,1344,-0.241694,0.104730




paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097897,0.637375
yes,0.338151,3321,0.068183,1.252560




paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


In [486]:
df_group

,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101797,0.622928
credit_card_(automatic),0.164339,1217,-0.105630,0.608733
electronic_check,0.455890,1893,0.185922,1.688682
mailed_check,0.193870,1305,-0.076098,0.718121


We see which feature could be important but we do not know if one categorical variable is more important than another (comparison of features)

## 3.6 Feature Importance: Mutual Information Score

A way to measrue the importance of categorical variables. HOw much do we learn about churn, when we have information about feature X

In [487]:
from sklearn.metrics import mutual_info_score

In [488]:
print(mutual_info_score(df_full_train.churn, df_full_train.contract))
print(mutual_info_score(df_full_train.churn, df_full_train.gender))
print(mutual_info_score(df_full_train.churn, df_full_train.partner))

0.0983203874041556
0.0001174846211139946
0.009967689095399745


In [489]:
#Because apply takes only functions with one argument need to wrap!
def mutual_info_churn_score(series):
    return mutual_info_score(series, df_full_train.churn)

In [490]:
#NEW METHOD Function to apply to each column or row.
df_full_train[categorical].apply(mutual_info_churn_score).sort_values(ascending=False)

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

## 3.7 Feature Importance: Correlation

How to find relevant NUMERICAL   variables!?

In [491]:
#NEW METHOD
df_full_train[numerical].corrwith(df_full_train.churn)

tenure           -0.351885
monthlycharges    0.196805
totalcharges     -0.196353
dtype: float64

In [492]:
df_full_train[df_full_train.tenure <= 2].churn.mean()

0.5953420669577875

In [493]:
df_full_train[df_full_train.tenure >= 2].churn.mean()

0.23594631394670298

## 3.8 One-hot encoding

Make categorical variables available for modelling, gender -> m/f becomes is_m --> 0/1 is_f --> 0/1

In [494]:
from sklearn.feature_extraction import DictVectorizer

In [495]:
df_train[['gender','contract']].iloc[:10].to_dict(orient = 'records')

[{'gender': 'female', 'contract': 'two_year'},
 {'gender': 'male', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'two_year'},
 {'gender': 'male', 'contract': 'month-to-month'},
 {'gender': 'male', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'month-to-month'},
 {'gender': 'female', 'contract': 'two_year'},
 {'gender': 'female', 'contract': 'month-to-month'}]

In [496]:
train_dicts = df_train[categorical + numerical].to_dict(orient = 'records')

In [497]:
dv = DictVectorizer(sparse = False)

In [498]:
#fit = recognize which are categorical  features, for each feature understand tha different value oiptions
#transform = each feature gets multiple columns with the falues inside the feture and the features themselves becom 0 and 1

X_train = dv.fit_transform(train_dicts)

In [499]:
val_dicts = df_val[categorical + numerical].to_dict(orient = 'records')

In [500]:
#Here no more fit necessary because we are using the same instance of the dict vectorizer
X_val = dv.transform(val_dicts)

## 3.9 Logistic Regression

In short: It is the same as linear regression but it takes the linear rigression and puts it into the sigmoid function which transforms every number so it fits into the range of 0 to 1. In Logistic Regression the linear regression part that is inputed into sigmoid is called score, then the output of sigmoid is the porbability

In [501]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

## 3.10 Training logistic regression with Scikit-Learn

In [502]:
from sklearn.linear_model import LogisticRegression

In [503]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [504]:
model.intercept_[0]

-0.10903521054753158

In [505]:
model.coef_[0].round(3)

array([ 0.475, -0.175, -0.408, -0.03 , -0.078,  0.063, -0.089, -0.081,
       -0.034, -0.073, -0.335,  0.316, -0.089,  0.004, -0.258,  0.141,
        0.009,  0.063, -0.089, -0.081,  0.266, -0.089, -0.284, -0.231,
        0.124, -0.166,  0.058, -0.087, -0.032,  0.07 , -0.059,  0.141,
       -0.249,  0.215, -0.12 , -0.089,  0.102, -0.071, -0.089,  0.052,
        0.213, -0.089, -0.232, -0.07 ,  0.   ])

In [506]:
#Hard predictions (gives you churn not churn)
model.predict(X_train)

array([0, 1, 1, ..., 1, 0, 1])

In [507]:
model.classes_

array([0, 1])

In [508]:
#Soft predictions (gives you the probability)
model.predict_proba(X_train)
#first value is probability of no churn, second is probability of churn

array([[0.90452229, 0.09547771],
       [0.32067632, 0.67932368],
       [0.36632527, 0.63367473],
       ...,
       [0.46838058, 0.53161942],
       [0.95745643, 0.04254357],
       [0.30127596, 0.69872404]])

In [509]:
y_pred = model.predict_proba(X_val)[:,1]

In [510]:
churn_decision = (y_pred >= 0.5)
churn_decision

array([False, False, False, ..., False,  True,  True])

In [511]:
#These are the customers that we predict that they will churn
df_val[churn_decision].customerid

3       8433-wxgna
8       3440-jpscl
11      2637-fkfsy
12      7228-omtpn
19      6711-fldfb
           ...    
1397    5976-jcjrh
1398    2034-cgrhz
1399    5276-kqwhg
1407    6521-yytyi
1408    3049-solay
Name: customerid, Length: 311, dtype: object

In [512]:
#the y that we have in the validation data
y_val

array([0, 0, 0, ..., 0, 1, 1])

In [513]:
#what our model predicted
churn_decision.astype(int)

array([0, 0, 0, ..., 0, 1, 1])

In [514]:
#Checking how much % is correctly predicted basicaly counting all Trues divided by the number of predictions
(y_val == churn_decision).mean()

0.8034066713981547

## 3.11 Model Interpretation

In [515]:
dv.feature_names_

['contract=month-to-month',
 'contract=one_year',
 'contract=two_year',
 'dependents=no',
 'dependents=yes',
 'deviceprotection=no',
 'deviceprotection=no_internet_service',
 'deviceprotection=yes',
 'gender=female',
 'gender=male',
 'internetservice=dsl',
 'internetservice=fiber_optic',
 'internetservice=no',
 'monthlycharges',
 'multiplelines=no',
 'multiplelines=no_phone_service',
 'multiplelines=yes',
 'onlinebackup=no',
 'onlinebackup=no_internet_service',
 'onlinebackup=yes',
 'onlinesecurity=no',
 'onlinesecurity=no_internet_service',
 'onlinesecurity=yes',
 'paperlessbilling=no',
 'paperlessbilling=yes',
 'partner=no',
 'partner=yes',
 'paymentmethod=bank_transfer_(automatic)',
 'paymentmethod=credit_card_(automatic)',
 'paymentmethod=electronic_check',
 'paymentmethod=mailed_check',
 'phoneservice=no',
 'phoneservice=yes',
 'seniorcitizen',
 'streamingmovies=no',
 'streamingmovies=no_internet_service',
 'streamingmovies=yes',
 'streamingtv=no',
 'streamingtv=no_internet_servic

In [516]:
model.coef_[0].round(3)

array([ 0.475, -0.175, -0.408, -0.03 , -0.078,  0.063, -0.089, -0.081,
       -0.034, -0.073, -0.335,  0.316, -0.089,  0.004, -0.258,  0.141,
        0.009,  0.063, -0.089, -0.081,  0.266, -0.089, -0.284, -0.231,
        0.124, -0.166,  0.058, -0.087, -0.032,  0.07 , -0.059,  0.141,
       -0.249,  0.215, -0.12 , -0.089,  0.102, -0.071, -0.089,  0.052,
        0.213, -0.089, -0.232, -0.07 ,  0.   ])

In [517]:
dict(zip(dv.feature_names_,model.coef_[0].round(3)))

{'contract=month-to-month': 0.475,
 'contract=one_year': -0.175,
 'contract=two_year': -0.408,
 'dependents=no': -0.03,
 'dependents=yes': -0.078,
 'deviceprotection=no': 0.063,
 'deviceprotection=no_internet_service': -0.089,
 'deviceprotection=yes': -0.081,
 'gender=female': -0.034,
 'gender=male': -0.073,
 'internetservice=dsl': -0.335,
 'internetservice=fiber_optic': 0.316,
 'internetservice=no': -0.089,
 'monthlycharges': 0.004,
 'multiplelines=no': -0.258,
 'multiplelines=no_phone_service': 0.141,
 'multiplelines=yes': 0.009,
 'onlinebackup=no': 0.063,
 'onlinebackup=no_internet_service': -0.089,
 'onlinebackup=yes': -0.081,
 'onlinesecurity=no': 0.266,
 'onlinesecurity=no_internet_service': -0.089,
 'onlinesecurity=yes': -0.284,
 'paperlessbilling=no': -0.231,
 'paperlessbilling=yes': 0.124,
 'partner=no': -0.166,
 'partner=yes': 0.058,
 'paymentmethod=bank_transfer_(automatic)': -0.087,
 'paymentmethod=credit_card_(automatic)': -0.032,
 'paymentmethod=electronic_check': 0.07,
 

In [518]:
#Creating a smaller model
small = ['contract', 'tenure', 'monthlycharges']

In [519]:
df_train[small].iloc[:10].to_dict(orient='records')

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1},
 {'contract': 'month-to-month', 'tenure': 4, 'monthlycharges': 30.5},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 75.1},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 70.3},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 19.75},
 {'contract': 'month-to-month', 'tenure': 6, 'monthlycharges': 109.9}]

In [520]:
dicts_train_small = df_train[small].to_dict(orient='records')
dicts_val_small = df_val[small].to_dict(orient='records')

In [521]:
dv_small = DictVectorizer(sparse=False)
dv_small.fit(dicts_train_small)

DictVectorizer(sparse=False)

In [522]:
dv_small.get_feature_names_out()


array(['contract=month-to-month', 'contract=one_year',
       'contract=two_year', 'monthlycharges', 'tenure'], dtype=object)

In [523]:
X_train_small = dv_small.transform(dicts_train_small)


In [524]:
model_small = LogisticRegression(solver='lbfgs')
model_small.fit(X_train_small, y_train)

LogisticRegression()

In [525]:
w0 = model_small.intercept_[0]
w0

-2.476775660276911

In [526]:
w = model_small.coef_[0]
w.round(3)

array([ 0.97 , -0.025, -0.949,  0.027, -0.036])

In [527]:
dict(zip(dv_small.get_feature_names_out(), w.round(3)))


{'contract=month-to-month': 0.97,
 'contract=one_year': -0.025,
 'contract=two_year': -0.949,
 'monthlycharges': 0.027,
 'tenure': -0.036}

In [528]:
-2.47 + (-0.949) + 30 * 0.027 + 24 * (-0.036)


-3.473

In [529]:
sigmoid(_)


0.030090303318277657

## 3.12 Using the model

In [530]:
dicts_full_train = df_full_train[categorical + numerical ].to_dict(orient = 'records')

In [531]:
dicts_full_train[:3]

[{'gender': 'male',
  'seniorcitizen': 0,
  'partner': 'yes',
  'dependents': 'yes',
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'no',
  'onlinesecurity': 'no_internet_service',
  'onlinebackup': 'no_internet_service',
  'deviceprotection': 'no_internet_service',
  'techsupport': 'no_internet_service',
  'streamingtv': 'no_internet_service',
  'streamingmovies': 'no_internet_service',
  'contract': 'two_year',
  'paperlessbilling': 'no',
  'paymentmethod': 'mailed_check',
  'tenure': 12,
  'monthlycharges': 19.7,
  'totalcharges': 258.35},
 {'gender': 'female',
  'seniorcitizen': 0,
  'partner': 'no',
  'dependents': 'no',
  'phoneservice': 'yes',
  'multiplelines': 'no',
  'internetservice': 'dsl',
  'onlinesecurity': 'yes',
  'onlinebackup': 'yes',
  'deviceprotection': 'yes',
  'techsupport': 'yes',
  'streamingtv': 'no',
  'streamingmovies': 'yes',
  'contract': 'one_year',
  'paperlessbilling': 'no',
  'paymentmethod': 'credit_card_(automatic)',
  'tenur

In [532]:
dv = DictVectorizer(sparse = False)
X_full_train = dv.fit_transform(dicts_full_train)

In [533]:
y_full_train =  df_full_train.churn.values

In [534]:
model = LogisticRegression()
model.fit(X_full_train,y_full_train)

LogisticRegression()

In [535]:
dicts_test = df_test[categorical + numerical].to_dict(orient = 'records')

In [536]:
X_test = dv.transform(dicts_test)

In [537]:
y_pred = model.predict_proba(X_test)[:,1]

In [538]:
churn_decision = (y_pred >= 0.5)

In [539]:
(churn_decision == y_test).mean()

0.815471965933286

In [541]:
customer = dicts_test[10]
customer

{'gender': 'male',
 'seniorcitizen': 1,
 'partner': 'yes',
 'dependents': 'yes',
 'phoneservice': 'yes',
 'multiplelines': 'no',
 'internetservice': 'fiber_optic',
 'onlinesecurity': 'no',
 'onlinebackup': 'yes',
 'deviceprotection': 'no',
 'techsupport': 'no',
 'streamingtv': 'yes',
 'streamingmovies': 'yes',
 'contract': 'month-to-month',
 'paperlessbilling': 'yes',
 'paymentmethod': 'mailed_check',
 'tenure': 32,
 'monthlycharges': 93.95,
 'totalcharges': 2861.45}

In [548]:
X_small = dv.transform([customer])

In [550]:
model.predict_proba(X_small)[0,1]

0.4056810977922081

In [551]:
y_test[10]

0